In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab or local 
%cd /content/drive/My\ Drive/SNLP\ Project

/content/drive/My Drive/SNLP Project


In [3]:
# list content of drive - verify you are where you are supposed to be
%ls

 amz_all_beauty/                    Lexical_analysis/
 amz_all_electronics/               meta_lstm_small_200dim.tsv
 CleantData_Apr-03-2020_01-31.zip   ModelResults/
 dataset_dumps.json                 ModelVisualization/
 Electronics_5.json.gz             'Project Ideas.gdoc'
 FinalModelResults/                'Project Proposal.gdoc'
'Final Report'/                    'Report MetaData'/
 glove.6B.200d.txt                 'Sentiment Analysis - Deep Learning.pptx'
 glove.6B.zip                       vecs_lstm_small_200dim.tsv


In [27]:
# list ensemble models here u wish to load
from tensorflow.keras.models import load_model

# set True or False
load_small = True
models = list()

if(load_small):
  small_elec_lstm = 'ModelResults/v3_767c02af219c4133946edcaf89e67387/model.h5'
  small_elec_cnn = 'ModelResults/v3_e5602b9bc9154c979a03dcf94e358d33/model.h5'
  small_elec_bilstm = 'ModelResults/v3_11ac8088df9a46e0b45b37b40157f115/model.h5'

  Train_ZipCSVFileName = 'amz_all_electronics/Data_Balanced_2000_Apr-03-2020_06-46/Train_2000_Apr-03-2020_06-46.zip'
  Test_ZipCSVFileName = 'amz_all_electronics/Data_Balanced_2000_Apr-03-2020_06-46/Test_2000_Apr-03-2020_06-46.zip'

  models.append(load_model(small_elec_lstm))
  models.append(load_model(small_elec_cnn))
  models.append(load_model(small_elec_bilstm))

  print('loaded models small')
else:
  mid_elec_lstm = 'ModelResults/v3_lstm_1f9ff1c3924d4bac8894dda0fc3bf5bf/model.h5'
  mid_elec_cnn = 'ModelResults/v3_e18b5532963e41bc8ec420d0ca77db0a/model.h5'
  #mid_elec_bilstm = 'ModelResults/v3_8495a3368e344e51bb84cbb01d445887/model.h5'
  mid_elec_bilstm = 'ModelResults/v3_bilstm_glove_739665f6a8794e98b874bec09723d536/model.h5'

  Train_ZipCSVFileName = 'amz_all_electronics/Data_Balanced_20000_Apr-03-2020_06-52/Train_20000_Apr-03-2020_06-52.zip'
  Test_ZipCSVFileName = 'amz_all_electronics/Data_Balanced_20000_Apr-03-2020_06-52/Test_20000_Apr-03-2020_06-52.zip'

  models.append(load_model(mid_elec_lstm))
  models.append(load_model(mid_elec_cnn))
  models.append(load_model(mid_elec_bilstm))

  print('loaded models medium')


loaded models small


In [28]:
# load your choice of dataset here . Specify paths as folder_datestring/file_datestring.zip

import pandas as pd
import numpy as np


df_train = pd.read_csv(Train_ZipCSVFileName)
df_train.info()

df_test = pd.read_csv(Test_ZipCSVFileName)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         10000 non-null  float64
 1   reviewTime      10000 non-null  object 
 2   reviewerID      10000 non-null  object 
 3   asin            10000 non-null  object 
 4   reviewText      10000 non-null  object 
 5   summary         10000 non-null  object 
 6   unixReviewTime  10000 non-null  int64  
 7   reviewText_len  10000 non-null  int64  
 8   summary_len     10000 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 703.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         1000 non-null   float64
 1   reviewTime      1000 non-null   object 
 2   reviewerID      1000 non-null   object 
 3  

In [29]:
# Get names of indexes for which column Age has value 30
index_neutrals_train = df_train[ df_train['overall'] == 3 ].index
index_neutrals_test = df_test[ df_test['overall'] == 3 ].index
 
# Delete these row indexes from dataFrame
df_train.drop(index_neutrals_train , inplace=True)
df_test.drop(index_neutrals_test , inplace=True)

df_train.loc[(df_train.overall == 1),'overall']= 0
df_train.loc[(df_train.overall == 2),'overall']= 0
df_train.loc[(df_train.overall == 4),'overall']= 1
df_train.loc[(df_train.overall == 5),'overall']= 1

df_test.loc[(df_test.overall == 1),'overall']= 0
df_test.loc[(df_test.overall == 2),'overall']= 0
df_test.loc[(df_test.overall == 4),'overall']= 1
df_test.loc[(df_test.overall == 5),'overall']= 1

df_train['reviewText_len'].describe()

count    8000.000000
mean      146.234875
std       109.121888
min         1.000000
25%        47.000000
50%       127.000000
75%       223.250000
max       399.000000
Name: reviewText_len, dtype: float64

In [30]:
# The maximum number of words to be used. (most frequent)
MAX_VOCAB_SIZE = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 200


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words= MAX_VOCAB_SIZE, filters='#$%&()*+<=>@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df_train['summary'].values + ' DELIM '+df_train['reviewText'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 8218 unique tokens.


In [31]:

X_test = tokenizer.texts_to_sequences(df_test['summary'].values + ' DELIM '+df_test['reviewText'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Y_Test = df_test['overall'].values
print('Shape of label tensor:', Y_Test.shape)

Shape of data tensor: (800, 250)
Shape of label tensor: (800,)


In [0]:
import numpy
from numpy import array
from numpy import argmax
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode

# make an ensemble prediction for multi-class classification
def ensemble_preds(members, testX):
	# make predictions
  ypred = list()

  for model in members:
    temp_pred= model.predict(testX)
    localPredArr = list()
    print(model.name)
    for local_pred in temp_pred:
      localPredArr.append(argmax(local_pred))
      
    ypred.append(localPredArr)

  print(ypred)
  ypred = array(ypred)
  # sum across ensemble members
  ypred = numpy.dstack(ypred)

  #take the innermost. ie only dimension
  ypred = ypred[0]
  
  result = [mode(predRow)[0][0] for predRow in ypred]
  return result
 
# evaluate a specific number of members in an ensemble
def run_models(members, Xtest, ylabels):	
	
	# make prediction
	ypred = ensemble_preds(members, Xtest)
	# calculate whole report here
	print(classification_report(ylabels, ypred, target_names=['Classified Neg','Classified Pos']))
	return accuracy_score(ylabels, ypred)

In [38]:
# evaluate n numbers of ensembles
score = run_models(models, X_test, Y_Test)
print('Ensemble Score for '+ str(len(models))+ ' models :', score)

sequential
sequential_1
sequential
[[0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 